## Data Wrangling: Clean, Transform, Merge, Reshape

In [1]:
import pandas as pd

## Combining and merging data sets

### Database-style DataFrame merges

In [2]:
df1 = pd.DataFrame({'data1' : range(5, 12), 'key' : list('bbacaab')})
df2 = pd.DataFrame({'data2' : range(56, 59), 'key' : list('abd')})
df1

,data1,key
0,5,b
1,6,b
2,7,a
3,8,c
4,9,a
5,10,a
6,11,b


In [3]:
df2

,data2,key
0,56,a
1,57,b
2,58,d


In [4]:
df1.merge(df2)

,data1,key,data2
0,5,b,57
1,6,b,57
2,11,b,57
3,7,a,56
4,9,a,56
5,10,a,56


By default, .merge() performs an [inner join](https://www.w3schools.com/sql/sql_join.asp) between the DataFrames, using the common columns as keys.

**What happens when there is no common column?**

In [5]:
dfa= pd.DataFrame({'data1' : range(5, 12), 'key1' : list('bbacaab')})
dfb = pd.DataFrame({'data2' : range(56, 59), 'key2' : list('abd')})

In [6]:
dfa.merge(dfb)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [8]:
dfa.merge(dfb,left_on='key1',right_on='key2')

,data1,key1,data2,key2
0,5,b,57,b
1,6,b,57,b
2,11,b,57,b
3,7,a,56,a
4,9,a,56,a
5,10,a,56,a


In [9]:
df1.merge(df2, how='outer')

,data1,key,data2
0,5.0,b,57.0
1,6.0,b,57.0
2,11.0,b,57.0
3,7.0,a,56.0
4,9.0,a,56.0
5,10.0,a,56.0
6,8.0,c,NaN
7,NaN,d,58.0


That means that it returns the cartesian product of the elements with common keys: if there are duplicates, it will return all the possible combinations:

In [11]:
df1

,data1,key
0,5,b
1,6,b
2,7,a
3,8,c
4,9,a
5,10,a
6,11,b


In [12]:
df3

,data2,key
0,56,a
1,57,b
2,58,d
3,59,b
4,60,d


In [13]:
df3 = pd.DataFrame({'data2' : range(56, 61), 'key' : list('abdbd')})
df1.merge(df3)

,data1,key,data2
0,5,b,57
1,5,b,59
2,6,b,57
3,6,b,59
4,11,b,57
5,11,b,59
6,7,a,56
7,9,a,56
8,10,a,56


In [14]:
df3

,data2,key
0,56,a
1,57,b
2,58,d
3,59,b
4,60,d


If the columns to join on don't have the same name, or we want to join on the index of the DataFrames, we'll need to specify that.

In [15]:
df4 = pd.DataFrame({'data2' : range(56, 61), 'rkey' : list('abdbd')})
df1.merge(df4)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [16]:
df1.merge(df4, left_on='key', right_on='rkey')

,data1,key,data2,rkey
0,5,b,57,b
1,5,b,59,b
2,6,b,57,b
3,6,b,59,b
4,11,b,57,b
5,11,b,59,b
6,7,a,56,a
7,9,a,56,a
8,10,a,56,a


If there are two columns with the same name that we do not join on, both will get transferred to the resulting DataFrame with a suffix. We can customize these suffixes.

In [17]:
df1.merge(df2, left_on='data1', right_on='data2', how='outer')

,data1,key_x,data2,key_y
0,5.0,b,NaN,NaN
1,6.0,b,NaN,NaN
2,7.0,a,NaN,NaN
3,8.0,c,NaN,NaN
4,9.0,a,NaN,NaN
5,10.0,a,NaN,NaN
6,11.0,b,NaN,NaN
7,NaN,NaN,56.0,a
8,NaN,NaN,57.0,b
9,NaN,NaN,58.0,d


In [18]:
df1.merge(df2, left_on='data1', right_on='data2', how='outer', suffixes=['_customer', '_order'])

,data1,key_customer,data2,key_order
0,5.0,b,NaN,NaN
1,6.0,b,NaN,NaN
2,7.0,a,NaN,NaN
3,8.0,c,NaN,NaN
4,9.0,a,NaN,NaN
5,10.0,a,NaN,NaN
6,11.0,b,NaN,NaN
7,NaN,NaN,56.0,a
8,NaN,NaN,57.0,b
9,NaN,NaN,58.0,d


### Merging on index

In [19]:
df4.index = range(5,10)
df4

,data2,rkey
5,56,a
6,57,b
7,58,d
8,59,b
9,60,d


In [20]:
df1

,data1,key
0,5,b
1,6,b
2,7,a
3,8,c
4,9,a
5,10,a
6,11,b


In [21]:
df1.merge(df4, left_on='data1', right_index=True)

,data1,key,data2,rkey
0,5,b,56,a
1,6,b,57,b
2,7,a,58,d
3,8,c,59,b
4,9,a,60,d


### Concatenating along an axis

In [22]:
pd.concat([df1, df2])

,data1,key,data2
0,5.0,b,NaN
1,6.0,b,NaN
2,7.0,a,NaN
3,8.0,c,NaN
4,9.0,a,NaN
5,10.0,a,NaN
6,11.0,b,NaN
0,NaN,a,56.0
1,NaN,b,57.0
2,NaN,d,58.0


In [24]:
df1

,data1,key
0,5,b
1,6,b
2,7,a
3,8,c
4,9,a
5,10,a
6,11,b


In [25]:
df2

,data2,key
0,56,a
1,57,b
2,58,d


In [23]:
pd.concat([df1, df2], axis=1)

,data1,key,data2,key
0,5,b,56.0,a
1,6,b,57.0,b
2,7,a,58.0,d
3,8,c,NaN,NaN
4,9,a,NaN,NaN
5,10,a,NaN,NaN
6,11,b,NaN,NaN


In [26]:
df4

,data2,rkey
5,56,a
6,57,b
7,58,d
8,59,b
9,60,d


In [23]:
pd.concat([df1, df4], axis=1)

,data1,key,data2,rkey
0,5.0,b,NaN,NaN
1,6.0,b,NaN,NaN
2,7.0,a,NaN,NaN
3,8.0,c,NaN,NaN
4,9.0,a,NaN,NaN
5,10.0,a,56.0,a
6,11.0,b,57.0,b
7,NaN,NaN,58.0,d
8,NaN,NaN,59.0,b
9,NaN,NaN,60.0,d


In [27]:
pd.concat([df1['data1'], df2['data2'], df3['data2']])

0     5
1     6
2     7
3     8
4     9
5    10
6    11
0    56
1    57
2    58
0    56
1    57
2    58
3    59
4    60
dtype: int64

In [28]:
pd.concat([df1['data1'], df2['data2'], df3['data2']], axis=1, keys=['x', 'y', 'z'])

,x,y,z
0,5,56.0,56.0
1,6,57.0,57.0
2,7,58.0,58.0
3,8,NaN,59.0
4,9,NaN,60.0
5,10,NaN,NaN
6,11,NaN,NaN


Not expected to be used often!

#### Digression

Attention! Be careful not to reassign to reserved words or functions- you will overwrite the variable.

In [26]:
pd.concat = [df1, df2] 
pd.concat([df3, df4]) 

TypeError: 'list' object is not callable

You can delete the overwritten variable, but you won't get back the original value. If it is an object or function from a module, you'll need to reload() the module, since Python doesn't load again an already imported module if you try to import it. reload() is useful also when you are actively developing your own module and want to load the latest definition of a function into memory.

In [27]:
del(pd.concat)
pd.concat([df3, df4]) 

AttributeError: module 'pandas' has no attribute 'concat'

In [28]:
import pandas as pd

pd.concat([df3, df4]) 

AttributeError: module 'pandas' has no attribute 'concat'

In [29]:
from importlib import reload

reload(pd)
pd.concat([df3, df4])

,data2,key,rkey
0,56,a,NaN
1,57,b,NaN
2,58,d,NaN
3,59,b,NaN
4,60,d,NaN
5,56,NaN,a
6,57,NaN,b
7,58,NaN,d
8,59,NaN,b
9,60,NaN,d


## Data transformation

### Removing duplicates

In [29]:
df1['key']

0    b
1    b
2    a
3    c
4    a
5    a
6    b
Name: key, dtype: object

In [30]:
df1['key'].duplicated()

0    False
1     True
2    False
3    False
4     True
5     True
6     True
Name: key, dtype: bool

In [31]:
df1[df1['key'].duplicated()]

,data1,key
1,6,b
4,9,a
5,10,a
6,11,b


In [32]:
df1[~df1['key'].duplicated()]

,data1,key
0,5,b
2,7,a
3,8,c


In [33]:
df1['key'].drop_duplicates()

0    b
2    a
3    c
Name: key, dtype: object

In [34]:
df1

,data1,key
0,5,b
1,6,b
2,7,a
3,8,c
4,9,a
5,10,a
6,11,b


In [36]:
df1.drop_duplicates('key')

,data1,key
0,5,b
2,7,a
3,8,c


In [37]:
df1

,data1,key
0,5,b
1,6,b
2,7,a
3,8,c
4,9,a
5,10,a
6,11,b


In [38]:
df1['duplicat'] = df1['key'].duplicated()

In [39]:
df1

,data1,key,duplicat
0,5,b,False
1,6,b,True
2,7,a,False
3,8,c,False
4,9,a,True
5,10,a,True
6,11,b,True


In [40]:
df1.drop_duplicates(subset='key', keep='last')

,data1,key,duplicat
3,8,c,False
5,10,a,True
6,11,b,True


In [41]:
import numpy as np

In [42]:
df_choice = pd.DataFrame({'searchID' : range(5, 12), 'bookingId' : list('bbacaab'),'delta':np.linspace(0,10,7)})

In [43]:
df_choice

,searchID,bookingId,delta
0,5,b,0.000000
1,6,b,1.666667
2,7,a,3.333333
3,8,c,5.000000
4,9,a,6.666667
5,10,a,8.333333
6,11,b,10.000000


In [44]:
df_choice.drop_duplicates(subset='bookingId')

,searchID,bookingId,delta
0,5,b,0.000000
2,7,a,3.333333
3,8,c,5.000000


### Renaming axis indexes

In [45]:
df1.index = list('bhkdsdf')
df1

,data1,key,duplicat
b,5,b,False
h,6,b,True
k,7,a,False
d,8,c,False
s,9,a,True
d,10,a,True
f,11,b,True


### Discretization and binning

In [46]:
import numpy as np

np.random.seed(42)

ages = pd.Series(np.random.randint(9,99, 50))
ages

0     60
1     23
2     80
3     69
4     29
5     91
6     95
7     83
8     83
9     96
10    32
11    11
12    30
13    61
14    10
15    96
16    38
17    46
18    10
19    72
20    68
21    29
22    41
23    84
24    66
25    30
26    97
27    57
28    67
29    50
30    68
31    88
32    23
33    70
34    70
35    55
36    70
37    59
38    63
39    72
40    11
41    59
42    15
43    29
44    81
45    47
46    26
47    12
48    97
49    68
dtype: int64

In [47]:
limits = [0, 14, 18, 35, 50, 65, 100]

pd.cut(ages, limits).value_counts()

(65, 100]    23
(18, 35]      9
(50, 65]      7
(0, 14]       5
(35, 50]      5
(14, 18]      1
dtype: int64

## String manipulation

### String object methods

In [48]:
bichos = pd.Series(np.random.choice(['Mantis Shrimp', 'Naked Mole Rat', 'Star Nosed Mole'],15))
bichos

0      Naked Mole Rat
1      Naked Mole Rat
2      Naked Mole Rat
3       Mantis Shrimp
4      Naked Mole Rat
5       Mantis Shrimp
6      Naked Mole Rat
7     Star Nosed Mole
8     Star Nosed Mole
9       Mantis Shrimp
10    Star Nosed Mole
11    Star Nosed Mole
12     Naked Mole Rat
13      Mantis Shrimp
14     Naked Mole Rat
dtype: object

In [49]:
bichos.upper()

AttributeError: 'Series' object has no attribute 'upper'

### Vectorized string functions in pandas

[Vectorized string functions in pandas](https://pandas.pydata.org/pandas-docs/stable/text.html) are grouped within the .str attribute of Series and Indexes. They have the same names as the regular Python string functions, but work on Series of strings.

In [50]:
bichos.str.upper()

0      NAKED MOLE RAT
1      NAKED MOLE RAT
2      NAKED MOLE RAT
3       MANTIS SHRIMP
4      NAKED MOLE RAT
5       MANTIS SHRIMP
6      NAKED MOLE RAT
7     STAR NOSED MOLE
8     STAR NOSED MOLE
9       MANTIS SHRIMP
10    STAR NOSED MOLE
11    STAR NOSED MOLE
12     NAKED MOLE RAT
13      MANTIS SHRIMP
14     NAKED MOLE RAT
dtype: object

In [51]:
bichos.str.lower()

0      naked mole rat
1      naked mole rat
2      naked mole rat
3       mantis shrimp
4      naked mole rat
5       mantis shrimp
6      naked mole rat
7     star nosed mole
8     star nosed mole
9       mantis shrimp
10    star nosed mole
11    star nosed mole
12     naked mole rat
13      mantis shrimp
14     naked mole rat
dtype: object

In [49]:
bichos.str.len()

0     14
1     14
2     14
3     13
4     14
5     13
6     14
7     15
8     15
9     13
10    15
11    15
12    14
13    13
14    14
dtype: int64

In [52]:
bichos.str.split()

0      [Naked, Mole, Rat]
1      [Naked, Mole, Rat]
2      [Naked, Mole, Rat]
3        [Mantis, Shrimp]
4      [Naked, Mole, Rat]
5        [Mantis, Shrimp]
6      [Naked, Mole, Rat]
7     [Star, Nosed, Mole]
8     [Star, Nosed, Mole]
9        [Mantis, Shrimp]
10    [Star, Nosed, Mole]
11    [Star, Nosed, Mole]
12     [Naked, Mole, Rat]
13       [Mantis, Shrimp]
14     [Naked, Mole, Rat]
dtype: object

In [53]:
bichos.str[:6]

0     Naked 
1     Naked 
2     Naked 
3     Mantis
4     Naked 
5     Mantis
6     Naked 
7     Star N
8     Star N
9     Mantis
10    Star N
11    Star N
12    Naked 
13    Mantis
14    Naked 
dtype: object

In [54]:
bichos.str.split().str[-1]

0        Rat
1        Rat
2        Rat
3     Shrimp
4        Rat
5     Shrimp
6        Rat
7       Mole
8       Mole
9     Shrimp
10      Mole
11      Mole
12       Rat
13    Shrimp
14       Rat
dtype: object